In [32]:
import sys
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import datetime as DT
print('system: ', sys.version)
print('numpy: ', np.__version__)
print('panda: ', pd.__version__)

system:  3.5.1 |Anaconda 2.4.1 (64-bit)| (default, Dec  7 2015, 15:00:12) [MSC v.1900 64 bit (AMD64)]
numpy:  1.10.1
panda:  0.17.1


In [2]:
# read in Indicator data into dataframe
parser = lambda date: pd.datetime.strptime(date, '%Y.%m.%d %H:%M:%S')
df_Ind = pd.read_csv('../data/150001_Indicators.csv', parse_dates=[0], date_parser=parser)
#rename 'Date' column to 'DateTime' and split apart
df_Ind.rename(columns = {'Date': 'DateTime'}, inplace=True)
temp = pd.DatetimeIndex(df_Ind['DateTime'])
df_Ind['Date'] = temp.date
df_Ind['Time'] = temp.time

In [3]:
# rearange to put 'Date' and 'Time' up front behind 'DateTime'
cols = df_Ind.columns.tolist()
cols = cols[0:1] + cols[-2:] + cols[1:-2]
#cols
df_Ind = df_Ind[cols]
df_Ind.head()

,DateTime,Date,Time,Ticket,Open/Close,BBAND(10),BBAND(20),BBAND(30),BBAND(40),BBAND(50),...,MACD(12) MAIN,MACD(12) SIGNAL,MACD(14) MAIN,MACD(14) SIGNAL,MACD(16) MAIN,MACD(16) SIGNAL,MACD(18) MAIN,MACD(18) SIGNAL,MACD(20) MAIN,MACD(20) SIGNAL
0,2015-01-02 08:00:00,2015-01-02,08:00:00,-1,ON_NewBar,1.209722,1.210053,1.210167,1.210231,1.210213,...,-0.000425,-0.000060,-0.000425,-0.000047,-0.000425,-0.000036,-0.000425,-0.000026,-0.000425,-0.000016
1,2015-01-02 08:01:00,2015-01-02,08:01:00,-1,ON_NewBar,1.209193,1.209771,1.209984,1.210093,1.210109,...,-0.000757,-0.000126,-0.000757,-0.000105,-0.000757,-0.000088,-0.000757,-0.000073,-0.000757,-0.000059
2,2015-01-02 08:02:00,2015-01-02,08:02:00,-1,ON_NewBar,1.208657,1.209489,1.209802,1.209957,1.210005,...,-0.001009,-0.000211,-0.001009,-0.000180,-0.001009,-0.000154,-0.001009,-0.000132,-0.001009,-0.000113
3,2015-01-02 08:03:00,2015-01-02,08:03:00,-1,ON_NewBar,1.208082,1.209193,1.209604,1.209807,1.209895,...,-0.001229,-0.000315,-0.001229,-0.000271,-0.001229,-0.000234,-0.001229,-0.000204,-0.001229,-0.000179
4,2015-01-02 08:04:00,2015-01-02,08:04:00,-1,ON_NewBar,1.207358,1.208821,1.209358,1.209620,1.209755,...,-0.001498,-0.000444,-0.001498,-0.000383,-0.001498,-0.000334,-0.001498,-0.000294,-0.001498,-0.000260


In [5]:
# read in order data
df_ord = pd.read_csv('../data/150001_Orders.csv', parse_dates=[1], date_parser=parser)

In [6]:
#rename 'Date' column to 'DateTime' and split apart
df_ord.rename(columns = {'Open Time': 'DateTime'}, inplace=True)
temp = pd.DatetimeIndex(df_ord['DateTime'])
df_ord['Date'] = temp.date
df_ord['Time'] = temp.time

import decimal
df_ord['DateTime_Sec'] = df_ord.DateTime.map(lambda x: (x-DT.datetime(1970,1,1)).total_seconds())

In [7]:
# rearange to put 'Date' and 'Time' up front behind 'DateTime'
cols = df_ord.columns.tolist()
cols = cols[1:2] + cols[-2:] + cols[0:1] + cols[2:-2]
#cols
df_ord = df_ord[cols]

#Convert 'DateTime' to timestamp
df_ord['DateTime'] = df_ord.DateTime.map(lambda x: DT.datetime.strptime(x, '%Y.%m.%d %H:%M:%S'))

In [8]:
df_ord.head()

,DateTime,Date,Time,Ticket,Type,Lots,Symbol,Open Price,StopLoss,TakeProfit,Close Time,Close Price,Commission,Swap,Profit,Comment,MagicNumber,Expiration
0,2015-01-04 23:41:30,2015-01-04,23:41:30,1,OP_SELL,0.1,EURUSD,1.19540,1.19940,1.19040,2015.01.05 00:15:40,1.19360,0,-0.15,18.0,Sell,150001,1970.01.01 00:00:00
1,2015-01-06 09:51:40,2015-01-06,09:51:40,2,OP_SELL,0.1,EURUSD,1.19392,1.19792,1.18892,2015.01.06 10:20:40,1.19293,0,0.00,9.9,Sell,150001,1970.01.01 00:00:00
2,2015-01-06 12:04:59,2015-01-06,12:04:59,3,OP_SELL,0.1,EURUSD,1.19122,1.19522,1.18622,2015.01.06 12:16:59,1.18954,0,0.00,16.8,Sell,150001,1970.01.01 00:00:00
3,2015-01-07 13:20:40,2015-01-07,13:20:40,4,OP_SELL,0.1,EURUSD,1.18432,1.18832,1.17932,2015.01.07 13:52:40,1.18318,0,0.00,11.4,Sell,150001,1970.01.01 00:00:00
4,2015-01-08 04:07:00,2015-01-08,04:07:00,5,OP_BUY,0.1,EURUSD,1.18198,1.17798,1.18698,2015.01.08 11:02:18,1.17798,0,0.00,-40.0,Buy[sl],150001,1970.01.01 00:00:00


In [9]:
# function returns sum of count of number of profit (+1) and loss trades (-1)
def rolling_profit_count(dataframe):
    count = 0
    for profit in dataframe:
        if (profit >= 0):
            count = count + 1
        else:
            count = count -1
    return count

In [18]:
# Add Rolling stats to Order DF
window = 15
df_ord['Roll_Profit_Count'] = pd.rolling_apply(df_ord['Profit'], window, rolling_profit_count,1)
df_ord['Roll_Mean'] = pd.rolling_mean(df_ord['Profit'], window)
df_ord['Roll_std'] = pd.rolling_std(df_ord['Profit'], window)
df_ord['Roll_var'] = pd.rolling_var(df_ord['Profit'], window)

In [19]:
# plot rolling data, intent was to determine range of UP, DOWN and FLAT for nex step
#plt.plot(df_ord['Ticket'], df_ord['Roll_std'])
#plt.plot(df_ord['Ticket'], df_ord['Profit'], marker='x', linestyle=':')

#plt.show()

In [20]:
# Create Trend Ranges, based on visual inspection of previous graph, and add Trends to Order dataframe
trend_range = [0, 6, 33, 60, 77, 86, 150, 171, 207, 222, 271, 314, 331, 348]
trend_labels = ['UP1', 'FLAT1', 'DOWN1', 'UP2', 'DOWN2', 'FLAT2', 'DOWN3', 'UP3', 'DOWN4', 'UP4', 
               'DOWN5', 'UP6', 'FLAT3']

df_ord['Trend'] = pd.cut(df_ord.Ticket, trend_range, labels=trend_labels).astype('category')

In [22]:
# Order Dataframe with Trends added
df_ord[['DateTime', 'Ticket', 'Profit', 'Trend', 'Roll_Profit_Count', 'Roll_Mean', 'Roll_std']].tail()

,DateTime,Ticket,Profit,Trend,Roll_Profit_Count,Roll_Mean,Roll_std
343,2015-12-14 13:14:24,344,-40,FLAT3,3,-0.853333,27.492515
344,2015-12-16 08:49:30,345,16,FLAT3,5,0.406667,27.823127
345,2015-12-16 18:59:05,346,-40,FLAT3,3,-4.906667,27.390078
346,2015-12-16 20:14:59,347,-40,FLAT3,3,-4.906667,27.390078
347,2015-12-17 00:01:30,348,16,FLAT3,3,-3.986667,27.873333


In [37]:
?map

In [43]:
# function to convert 'DateTime' to seconds
seconds = lambda x: (x - DT.datetime(1970, 1, 1)).total_seconds()

# Convert Date time in Order DF to seconds and pull that Series out
df_ord['DateTime_sec'] = df_ord['DateTime'].apply(seconds)

In [44]:
# pull 'Times' of trend range changes from order dataframe
# i.e. if 'Ticket' is in trend_range list, return the 'DateTime'
trend_range_DT = df_ord[df_ord['Ticket'].isin(trend_range)]['DateTime_sec']

In [ ]:
# Convert Date time in Indicator DF to seconds and pull that Series out
df_Ind['DateTime_sec'] = df_Ind['DateTime'].apply(seconds)

In [46]:
df_Ind['Trend'] = pd.cut(df_Ind.DateTime_sec, trend_range_DT, labels=trend_labels).astype('category')

AttributeError: 'DataFrame' object has no attribute 'DateTime_sec'

trend_range_DT2 = map(lambda x: DT.datetime.fromtimestamp(float(x)), trend_range_DT)

trend_range_DT_Dec = map(lambda x: decimal.Decimal(x), trend_range_DT2)

trend_range_DT_Dec

--- Confirm data types

trend_range_DT_dec

type(trend_range_DT[0])

type(df_Ind['DateTime'][0])

In [140]:
df_ord['DateTime_dec'] = df_ord['DateTime'].map(lambda x: DT.timedelta(x - DT.datetime(1970, 1, 1, 0, 0, 0)))

TypeError: unsupported type for timedelta days component: Timedelta

In [138]:
df_ord.head()

,DateTime,Date,Time,Ticket,Type,Lots,Symbol,Open Price,StopLoss,TakeProfit,...,Profit,Comment,MagicNumber,Expiration,Roll_Profit_Count,Roll_Mean,Roll_std,Roll_var,Trend,DateTime_dec
0,2015-01-04 23:41:30,2015-01-04,23:41:30,1,OP_SELL,0.1,EURUSD,1.19540,1.19940,1.19040,...,18.0,Sell,150001,1970.01.01 00:00:00,1,18.000,NaN,NaN,UP1,<map object at 0x000000004F094908>
1,2015-01-06 09:51:40,2015-01-06,09:51:40,2,OP_SELL,0.1,EURUSD,1.19392,1.19792,1.18892,...,9.9,Sell,150001,1970.01.01 00:00:00,2,13.950,5.727565,32.8050,UP1,<map object at 0x000000004F094908>
2,2015-01-06 12:04:59,2015-01-06,12:04:59,3,OP_SELL,0.1,EURUSD,1.19122,1.19522,1.18622,...,16.8,Sell,150001,1970.01.01 00:00:00,3,14.900,4.371499,19.1100,UP1,<map object at 0x000000004F094908>
3,2015-01-07 13:20:40,2015-01-07,13:20:40,4,OP_SELL,0.1,EURUSD,1.18432,1.18832,1.17932,...,11.4,Sell,150001,1970.01.01 00:00:00,4,14.025,3.975236,15.8025,UP1,<map object at 0x000000004F094908>
4,2015-01-08 04:07:00,2015-01-08,04:07:00,5,OP_BUY,0.1,EURUSD,1.18198,1.17798,1.18698,...,-40.0,Buy[sl],150001,1970.01.01 00:00:00,3,3.220,24.404754,595.5920,UP1,<map object at 0x000000004F094908>


In [118]:
def timesort(dt, tDT, tDT_L):
    
    tDTa = list(tDT.sort(reverse=True))
    print(type(tDT))
    for t in tDT:
        if (dt < t):
            return(tDT_L[t.index -1])

In [119]:
df_Ind['Trend'] = df_Ind.DateTime.apply(timesort(['DateTime'], trend_range_DT, trend_labels))

TypeError: 'NoneType' object is not iterable